In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from IPython.display import display
from PIL import Image

class compact(nn.Module):

    def __init__(self, num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=16, upscale=2, act_type='prelu', **kwargs):
        super(compact, self).__init__()
        self.num_in_ch = num_in_ch
        self.num_out_ch = num_out_ch
        self.num_feat = num_feat
        self.num_conv = num_conv
        self.upscale = upscale
        self.act_type = act_type

        self.body = nn.ModuleList()
        # the first conv
        self.body.append(nn.Conv2d(num_in_ch, num_feat, 3, 1, 1))
        # the first activation
        if act_type == 'relu':
            activation = nn.ReLU(inplace=True)
        elif act_type == 'prelu':
            activation = nn.PReLU(num_parameters=num_feat)
        elif act_type == 'leakyrelu':
            activation = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.body.append(activation)

        # the body structure
        for _ in range(num_conv):
            self.body.append(nn.Conv2d(num_feat, num_feat, 3, 1, 1))
            # activation
            if act_type == 'relu':
                activation = nn.ReLU(inplace=True)
            elif act_type == 'prelu':
                activation = nn.PReLU(num_parameters=num_feat)
            elif act_type == 'leakyrelu':
                activation = nn.LeakyReLU(negative_slope=0.1, inplace=True)
            self.body.append(activation)

        # the last conv
        self.body.append(nn.Conv2d(num_feat, num_out_ch *
                         upscale * upscale, 3, 1, 1))
        # upsample
        self.upsampler = nn.PixelShuffle(upscale)

    def forward(self, x):
        out = x

        out = self.body[0](out)
        out = self.body[1](out)
        # out = self.body[2](out)
        # out = self.body[3](out)
        # out = self.body[4](out)
        # out = self.body[5](out)
        out = out[:, :3, :, :]
        
        # out = out * 5 + 0.5

        # for i in range(0, len(self.body)):
        #     out = self.body[i](out)
        # out = self.upsampler(out)
        # base = F.interpolate(x, scale_factor=self.upscale, mode='nearest')
        # out += base
        return torch.clamp(out, min=0, max=1)

to_pil = torchvision.transforms.ToPILImage()
to_tensor = torchvision.transforms.ToTensor()


device = torch.device("cuda")
model = compact(num_feat=48, num_conv=8).to(device).half()
model.eval()
model.load_state_dict(torch.load("R:/AnimeJaNai_UC.pth", map_location=device)['params'])
# model.load_state_dict(torch.load("E:/project/neosr/experiments/compact_small/models/net_g_latest.pth", map_location=device)['params'])
# model.load_state_dict(torch.load("E:/project/neosr/experiments/compact_test/models/net_g_latest.pth", map_location=device)['params'])
image2 = to_tensor(Image.open("C:/Users/khoi/Videos/test.png").convert("RGB")).unsqueeze(0).half().to(device)
out = model(image2)[0]
print(out.shape)
# display(to_pil(out[:3, :, :]))
display(to_pil(out))
# out1 = out[0, :, :]
# out2 = out[1, :, :]
# out3 = out[2, :, :]
# out4 = out[3, :, :]
# display(to_pil(out1))
# display(to_pil(out2))
# display(to_pil(out3))
# display(to_pil(out4))



In [8]:
print(3024/1920)
# 1.575
# 94

1.575
120.0


In [47]:
print((3024-2560)/2)
print((1890-1440)/2)
print((3024-1280)/2)
print((1890-720)/2)

232.0
225.0
872.0
585.0
